In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
tf.compat.v1.disable_eager_execution()

In [3]:
data = pd.read_csv('train.tsv', delimiter='\t').values
train_data, val_data = train_test_split(data, random_state=117)

In [4]:
train_queries_cnt = len(np.unique(train_data[:, 2]))
val_queries_cnt = len(np.unique(val_data[:, 2]))

In [5]:
def get_proba(scores):
    return tf.nn.softmax(scores)


def loss(y_true, y_pred):
     return -tf.reduce_sum(get_proba(y_true) * tf.log(get_proba(y_pred) + 1e-10))

In [7]:
def generate_batches(data):
    query_to_inds = defaultdict(list)
    for ind, row in enumerate(data):
        query_to_inds[row[2]].append(ind)
    inds_batches = list(query_to_inds.values())
    while True:
        inds = np.random.choice(inds_batches)
        yield data[inds, 3:], data[inds, 1]

In [8]:
model = keras.models.Sequential([
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dense(1),
])

model.compile(keras.optimizers.Adam(), loss=loss)

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
model.fit_generator(
    generate_batches(train_data),
    steps_per_epoch=train_queries_cnt,
    epochs=100,
    # validation_data=generate_batches(val_data),
    # validation_steps=val_queries_cnt,
)

RuntimeError: You must compile your model before using it.